In [ ]:
!pip install xlrd
!pip install fuzzywuzzy

import pandas as pd
import numpy as np
import re
from fuzzywuzzy import process

In [ ]:
df = pd.read_excel("/content/GSAF5.xls")    # Downloading dataset via xls file

### Initial examination of dataset

In [ ]:
df   # First impression of the dataset

In [ ]:
df.shape    # Viewing the columns & rows of the dataset

In [ ]:
df.describe()   # Gaining information about the count, mean, min, max & standard deviation (only for numerical columns)

In [ ]:
df.nunique()    # Getting unique values within the dataset

In [ ]:
df.dtypes # Checking the data types of each column

In [ ]:
df.isna().sum() # Checking for missing values

### Dealing with column names & dropping irrelevant ones

In [ ]:
df.columns = df.columns.str.lower().str.replace(' ', '_')   #Changing the column names to be lower case & an "_" instead of a space.

df.rename(columns = {'species_' : 'species'}, inplace = True) # Fixing species column that had a space at the end

In [ ]:
# Deleted unnecessary columns from the dataset: (We deemed them unnecessary due to our hypothesis or because they were mostly null)
df.drop(columns = ['unnamed:_21', 'unnamed:_22', 'year', 'pdf', 'href_formula', 'href', 'case_number', 'case_number.1', 'original_order'], inplace = True)


In [ ]:
df.shape

In [ ]:
df.dtypes # Noticed here age is a string so ammending it

df.age = pd.to_numeric(df.age, errors = 'coerce') # Converting column to numeric, adding in errors = coerce part to convert non numerical values to nulls and avoid an error.

df.dtypes # Confirming it worked

### Dealing with null values & duplicates

In [ ]:
"""Cleaning up some unique values in some columns for improved accuracy"""

df.type = df.type.replace({' Provoked': 'Provoked', 'Questionable': np.nan,'Watercraft': 'Unprovoked', 'Sea Disaster': np.nan, '?': np.nan, 'Unconfirmed': np.nan, 'Unverified': np.nan, 'Invalid': np.nan, 'Boat': 'Unprovoked' })
print(df.type.unique()) # Leave 'provoked', 'unprovoked', 'under investigation'


df.sex = df.sex.replace({'lli': np.nan, '.': np.nan, 'N': np.nan, 'M ': 'M', ' M': 'M', 'M x 2': 'M'})
print(df.sex.unique()) # Leave 'M,' 'F'

df['fatal_y/n'] = df['fatal_y/n'].replace({'F': 'Y', 'M': np.nan, 'n': 'N', 'Nq': 'N',
                             'UNKNOWN': np.nan, 2017: np.nan, 'Y x 2': 'Y',
                             ' N': 'N', 'N ': 'N', 'y': 'Y'}) # Leave 'Y', 'N'
df['fatal_y/n'].unique()

df.country.unique() # Noticing quite a few countries with a "?" at the end so i'll remove them

df.country = df.country.str.rstrip("?") # Using strip to remove those with a ? and keep the rest.

df.species = df.species.str.lower()
df.species = df.species.str.strip()
  # remove all capital letters and trailing spaces

df.species.nunique()

In [ ]:
df = df.dropna(subset=['fatal_y/n', 'type'], how='all') # Dropping rows with fatal & type that are null as they don't add value to Hypothesis 1
df = df.dropna(subset=['fatal_y/n', 'species'], how='all')  # Dropping rows with fatal & species that are null as they don't add value to Hypothesis 2
df = df.dropna(subset=['name', 'sex'], how='all') # Dropping 70 rows that had both name & sex null so we can fill them in later on.

In [ ]:
# Using regex to fix data column format:

df["date"] = df["date"].apply(lambda x: re.findall(r"\b(?:\d{4}-\d{2}-\d{2}|\d{2}-[A-Za-z]{3}-\d{4})\b", str(x))[0] if len(re.findall(r"\b(?:\d{4}-\d{2}-\d{2}|\d{2}-[A-Za-z]{3}-\d{4})\b", str(x))) > 0 else np.NaN)

df.date.isna().sum()  # Now created 853 nulls due to adjusting the format

df.date.fillna('Unverified', inplace = True)  # Filling nulls with unverified as we don't want to lose 853 rows in our data.

In [ ]:
# Filling in null names with John or Jane Doe, as a better way to display the data than marking unknown.

df.loc[(df.sex == "M") & (df.name.isna()), 'name'] = 'John Doe'
df.loc[(df.sex == "F") & (df.name.isna()), 'name'] = 'Jane Doe'

df.loc[df.name.isin(['John Doe', 'Jane Doe'])]  # Checking that it worked.

In [ ]:
""" Filling in remaining nulls in the columns. Some we replaced with just unknowns as its not very relevant to our hypothesis and others we used aggregation to fill."""

df.dropna(subset = ['type'], inplace = True) # Dropping all rows with type as null to not mess with Hypothesis 1
df.country.fillna('Unknown', inplace = True)   # Only 50 null countries so marking unknown.
df.state.fillna('Not Confirmed', inplace = True)  # Marking the state nulls as not confirmed as they are also irrelevant to our hypothesis
df.location.fillna('Unknown', inplace = True) # Marking location nulls as unknown to not contaminate dataset.
df.activity.fillna('Unknown', inplace = True) # Marking missing activities as unknown.
df.sex = df.sex.fillna(df.sex.mode()[0])  # Filling in the null sex values with the mode, shouldn't impact our hypothesis at all.
df.age = df.age.fillna(df.age.mean()) # Filling in the null age values with the mean, to not affect the data too much or affect our hypothesis.
df.injury.fillna('Not Specified', inplace = True) # Filling in only a handfull of null injuries with not specified
df.dropna(subset = ['fatal_y/n'], inplace = True) # Dropping only 29 rows with fatality as null because it is very important for both of our hypothesis.
df.time.fillna('Not Specified', inplace = True)   # Filling in the time null values with unknowns as there are over 2807 null rows and we can't lose that much of our data.
df.source.fillna('Unconfirmed', inplace = True) # Filling in the nulls of source with unconfirmed as it is not very relevant to our hypothesis.

In [ ]:
df.isna().sum() # We will clean species once we start hypothesis 2, we don't want to lose 2776 rows for hypothesis 1

In [ ]:
df.duplicated().sum() # Found there are 2 duplicate rows
df.drop_duplicates(inplace = True)  # Dropping both rows as they are irrelevant
df.duplicated().sum() # Checking its 0

### Hypothesis 1: Provoked shark attacks are more fatal than unprovoked attacks.

In [ ]:
# Hypothesis 1: Provoked shark attacks are more deadly than unprovoked

df_hypothesis_1 = df[['name','sex','type','fatal_y/n']] # Created new data frame specific to hypothesis 1 with only relevant columns for us.

In [ ]:
# Checking for remaining null values in type & fatal

print("Remaining Null Values in 'type' column:", df_hypothesis_1['type'].isna().sum())
print("Remaining Null Values in 'fatal_y/n' column:", df_hypothesis_1['fatal_y/n'].isna().sum())

In [ ]:
df_hypothesis_1 = df_hypothesis_1.reset_index(drop=True)    # Resetting index after dropping values.

df_hypothesis_1

In [ ]:
grouped_hypothesis_1 = df_hypothesis_1.groupby(['type', 'fatal_y/n']).size().unstack(fill_value=0)  # Grouping by type & fatal result to see our results.

grouped_hypothesis_1['fatality_rate'] = round(grouped_hypothesis_1['Y'] / (grouped_hypothesis_1['Y'] + grouped_hypothesis_1['N']) * 100, 2).astype(str) + '%'   # Adding in a column for fatality rate to test our theory.

grouped_hypothesis_1  # Seeing final result --> Hypothesis 1 is wrong in the end.

In [ ]:
"""Visualising data using a bar chart"""

# Create a new DataFrame to visualise by copying grouped_hypothesis_1 to make some changes to it without altering original.
visual_hypothesis_1 = grouped_hypothesis_1.copy()

# Changing the fatality rate to a float and removing % sign to use for bar chart
visual_hypothesis_1['fatality_rate'] = visual_hypothesis_1['fatality_rate'].str.rstrip("%").astype(float)

# Excluding 'Under investigation' type to make the data more clean:
visual_hypothesis_1 = visual_hypothesis_1.drop(index='Under investigation')

# Calculating the inverse (non-fatal percentage) in order to make it a stacked bar chart.
visual_hypothesis_1['non_fatal_rate'] = 100 - visual_hypothesis_1['fatality_rate']

# Plotting stacked bar chart with fatality rate as the y axis & attack type is the x axis.
visual_hypothesis_1[['fatality_rate', 'non_fatal_rate']].plot(kind='bar', stacked=True, figsize=(9, 5), title="Fatality Rate by Attack Type", ylabel="Fatality Rate (%)", xlabel="Attack Type", color=['red', 'blue'])

### Hypothesis 2: Great whites are the most fatal

In [ ]:
# Hypothesis 2: Great white are the deadliest shark species in terms of total fatalities

# Check out the species column. We see lots of unique values
print(df.species.value_counts())
print(df.species.nunique())

In [ ]:
# Create a baby dataframe to compare species and fatal_y/n

df2 = df[['species', 'fatal_y/n']]
df2

In [ ]:
print(df2.species.isna().sum()) # Check for nulls
df2 = df2.dropna() # Drop nulls
df2

In [ ]:
# remove 'not stated' from species

df2 = df2[df2['species'] != 'not stated']
df2.reset_index(drop=True, inplace=True) # Reset index
df2 # 6 rows removed

In [ ]:
# To further clean up the column, we will use fuzzy matching and exact matching.

# First, create a list of valid species names.

# Found a list of 35 species known to have been involved in shark attacks online
# (https://www.floridamuseum.ufl.edu/shark-attacks/factors/species-implicated/).

valid_species = ['grey reef', 'bronze whaler', 'spinner', 'silky', 'galapagos', 'bull', 'blacktip',
'oceanic whitetip', 'blacktip reef', 'dusky', 'caribbean reef', 'sandbar', 'requiem', 'sand tiger',
'white', 'tiger', 'tope', 'nurse', 'port jackson', 'cookiecutter', 'shortfin mako', 'mako', 'porbeagle',
'lemon', 'sevengill', 'spotted wobbegong', 'ornate wobbegong', 'wobbegong', 'blue', 'guitarfish',
'hammerhead', 'whitetip reef', 'atlantic angel', 'leopard'] # Removed 'shark' to help with fuzzy matching (everything would match with 'shark')

# Then, we fuzzy match

# Create fuzzy matching function
def fuzzy_match_for_species(species_name):
    if species_name is not None:  # If the species name is not None (not NaN)
        best_match, score = process.extractOne(species_name, valid_species) # finds the value from valid_species most similar to the given string
        if score >= 80:  # Fuzzy match with threshold of 80
            return best_match
    return species_name  # Return the original species name if no match

# Apply fuzzy matching to species column and add fuzzy match column
df2['fuzzy_match'] = df2['species'].apply(fuzzy_match_for_species)

df2

In [ ]:
df2.fuzzy_match.nunique() # Still plenty of unique values, but no longer 1500

In [ ]:
# Now, find exact matches in the fuzzy_match column

# Create function

def check_exact_match(species):
    for valid_species_name in valid_species:
        if valid_species_name in species:  # Check if the valid species name is in the string
            return valid_species_name
    return None  # Return None if no exact match found

# Step 4: Apply exact match check to fuzzy matches
df2['match'] = df2['fuzzy_match'].apply(check_exact_match)

df2



In [ ]:
df2.match.isna().sum() # No match found for 1230 values

In [ ]:
df2 = df2.dropna() # drop the nulls
df2 = df2.reset_index(drop=True)  # Reset the index
df2

In [ ]:
# Now to figure out which species have the most fatalities!

fatalities_per_species = df2.groupby('match')['fatal_y/n'].apply(lambda x: (x == 'Y').sum())
fatalities_per_species = fatalities_per_species[fatalities_per_species > 0] # exclude non-fatal species
# Sort the species by the count of 'Y' (fatalities), from high to low
fatalities_per_species = fatalities_per_species.sort_values(ascending=False)
fatalities_per_species

In [ ]:
# Bar chart for the 11 fatal species
print(type(fatalities_per_species))
# fatalities_per_species = fatalities_per_species[fatalities_per_species['fatal_y/n']!= 0]

fatalities_per_species.plot(kind='barh', x='match', y='fatal_y/n', title='Fatal attacks per species', figsize=(10, 6), color='skyblue')

